# Importação de dados:

In [ ]:
import os
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

### Baixar recursos do NLTK (apenas na primeira execução)

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

### Função para extrair texto de PDFs

In [ ]:
def pdf_para_txt(caminho_pdf):
	with open(caminho_pdf, 'rb') as f:
		leitor = PyPDF2.PdfReader(f)
		texto = ""
		for pagina in range(len(leitor.pages)):
			texto += leitor.pages[pagina].extract_text()
	return texto

### Diretórios com os PDFs

In [ ]:
diretorios = {
	'poesia': 'pdfs/poesia/',
	'prosa': 'pdfs/prosa/',
	'jornalismo': 'pdfs/jornalismo/'
}

### Função para limpar e remover stopwords

In [ ]:
def limpar_texto(texto):
	stop_words = set(stopwords.words('english'))
	palavras = word_tokenize(texto.lower())
	palavras_limpa = [palavra for palavra in palavras
					if palavra.isalnum()
					and palavra not in stop_words]
	return " ".join(palavras_limpa)

### Extraindo textos e gerando classes

In [ ]:
textos = []
classes = []

for classe, caminho in diretorios.items():
	for arquivo in os.listdir(caminho):
		if arquivo.endswith('.pdf'):
			texto = pdf_para_txt(os.path.join(caminho, arquivo))
			texto_limpo = limpar_texto(texto)
			textos.append(texto_limpo)
			classes.append(classe)

### Criando a matriz Bag of Words

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(textos)
print("Matriz BoW: ", X.toarray())
print("Vocabulário: ", vectorizer.get_feature_names_out())
print("Classes: ", classes)

### Divisão dos dados em treino e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size=0.3, random_state=42)